In [1]:
# Makes plots / images embedded in the file
%matplotlib inline

In [4]:
from zipfile import ZipFile
from urllib.request import urlopen
import numpy as np

#URL = 'https://uwmadison-cs534-term-project-f2018-cdh.github.io/assets/datasets/entities.zip'

#url = urlopen(URL)
#output = open('environment.zip','wb')
#output.write(url.read())
#output.close()

zip_ref = ZipFile('entities.zip', 'r')
zip_ref.extractall('./dataset')
zip_ref.close()

In [5]:
# Import data and keras layers

from keras import backend as k
from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, UpSampling2D, Dense, Reshape
from keras.layers import Flatten, Dropout, BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from tqdm import tqdm
import matplotlib.pyplot as plt

ImportError: cannot import name 'get_config' from 'tensorflow.python.eager.context' (C:\Users\Alex\anaconda3\lib\site-packages\tensorflow\python\eager\context.py)

In [4]:
import os

files = os.listdir('./dataset/entities')
for name in files:
    print(name)

entities_label.npy
entities_image.npy


In [0]:
x1_train = np.load('./dataset/entities/entities_image.npy')
y1_train = np.load('./dataset/entities/entities_label.npy')

x1_test = np.array([])
y1_test = np.array([])

#Optimizer
adam = Adam(lr=2e-4, beta_1=0.5)

In [14]:
# Creating the generator
generator = Sequential()
generator.add(Dense(128*8*8, input_dim = 100))
generator.add(Activation('relu'))
generator.add(Dense(128*8*8))
generator.add(Activation('relu'))
generator.add(BatchNormalization())
generator.add(Reshape((8, 8, 128)))
generator.add(UpSampling2D((2,2)))
generator.add(Conv2D(64, kernel_size = (5, 5), padding = 'same'))
generator.add(Activation('relu'))
generator.add(BatchNormalization())
generator.add(UpSampling2D((2,2)))
generator.add(Conv2D(4, kernel_size = (5, 5), padding = 'same'))
generator.add(Activation('tanh'))

print(generator.summary())
generator.compile(loss = 'binary_crossentropy', optimizer = adam)  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 8192)              827392    
_________________________________________________________________
activation_4 (Activation)    (None, 8192)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 8192)              67117056  
_________________________________________________________________
activation_5 (Activation)    (None, 8192)              0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 8192)              32768     
_________________________________________________________________
reshape_2 (Reshape)          (None, 8, 8, 128)         0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 16, 16, 128)       0         
__________

In [15]:
# Creating the discriminator
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size = (5, 5), strides = (2, 2), padding = 'same', input_shape = (32, 32, 4)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Conv2D(128, kernel_size = (5, 5), strides = (2, 2), padding = 'same'))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Flatten())
discriminator.add(Dropout(0.5))
discriminator.add(Dense(1, activation = 'sigmoid'))
print(discriminator.summary())
discriminator.compile(loss = 'binary_crossentropy', optimizer = adam) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 16, 16, 64)        6464      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 8, 8, 128)         204928    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 8192)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 8193      
Total para

In [0]:
# Create the GAN
discriminator.trainable = False
gan = Sequential()
gan.add(generator)
gan.add(discriminator)
gan.compile(loss = 'binary_crossentropy', optimizer = adam)

In [0]:
# Visualize generated data
def plotGeneratedImages():
  generated_images = generator.predict(noise)
  fig = plt.figure(figsize=(9, 9))        
  for i in range(9):
    plt.subplot(3, 3, i+1)
    img = np.clip(generated_images[i, :],0,1)
    img = img.reshape((32, 32, 4))
    plt.tight_layout()
    plt.imshow(img)
    plt.axis('off')

In [0]:
epochs = 1000
batch_size = 32

batch_count = int(x1_train.shape[0] / batch_size)

for epoch in range(epochs):

    for index in tqdm(range(batch_count)):
        
        # Real images
        image_batch = x1_train[index * batch_size: (index + 1) * batch_size]
        
        # Get initial random input for generator
        noise = np.random.uniform(-1, 1, size=[batch_size, 100])
        # Generating images
        generated_data = generator.predict_on_batch(noise)

        X = np.concatenate([generated_data, image_batch])
        y = np.zeros(2 * batch_size)
        y[:batch_size] = 1

        # Train discriminator
        discriminator.trainable = True
        discriminator.train_on_batch(x=X, y=y)

        # Train generator 
        y = np.zeros(batch_size)
        discriminator.trainable = False
        gan.train_on_batch(x=noise, y=y)
        
        img = generated_images = generator.predict(noise)   
        
    
plotGeneratedImages()


 14%|█▍        | 12/86 [00:02<00:14,  5.25it/s]

In [0]:
noise = np.random.uniform(-1, 1, size=[batch_size, 100])
plotGeneratedImages()

In [0]:
from google.colab import files
import matplotlib.image as mpimg
import uuid

images = 20
noise = np.random.uniform(-1, 1, size=[images, 100])
generated_images = generator.predict(noise)    
for i in range(images):
  name = str(uuid.uuid4().hex) + '.png'
  img = np.reshape(np.clip(generated_images[i, :],0,1),(32, 32, 4))
  mpimg.imsave(name, img)             
  files.download(name)
